<a href="https://colab.research.google.com/github/crystdang/Amazon-Vine-Analysis/blob/main/Amazon_Reviews_ETL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.2 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.3'
spark_version = 'spark-3.2.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:14 http://ppa.launchpad.net

In [2]:
pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 20.2 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=10f2d46aaaaee93b1ecf0e16a01bda23bb59bf63b9f13106a0ae2444fc5ba322
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


In [3]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2023-01-04 00:29:11--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  5.93MB/s    in 0.2s    

2023-01-04 00:29:11 (5.93 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M17-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [5]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Tools_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   15785389|R2UM5QMHBHC90Q|B00H5U9ZD6|     115362950|WallPeg 12 sq ft ...|           Tools|          5|            0|          0|   N|                Y|Great organizer, ...|Very nice. Will o...| 2015-08-31|
|         US|   47910848|  RF0D1LEIF6L7|B001TJGCS0|     570955425|Nite Ize Nite Daw...|           Tools|          4|    

### Create DataFrames to match tables

In [6]:
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame


In [7]:
from pyspark.sql.functions import sum,avg,max,count
# Create the customers_table DataFrame
customers_df = df.groupBy("customer_id").agg(count("*").alias("customer_count")).withColumnRenamed("", "customer_count")
customers_df.show(truncate=False)

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|45978717   |2             |
|43622307   |1             |
|740678     |4             |
|29045703   |1             |
|52484548   |3             |
|42560427   |4             |
|17067926   |1             |
|10093406   |1             |
|44979559   |1             |
|19432125   |1             |
|26079415   |1             |
|29931671   |1             |
|12945150   |3             |
|12036434   |1             |
|14230926   |1             |
|45015535   |1             |
|39064792   |1             |
|20709090   |1             |
|45074906   |1             |
|26767269   |2             |
+-----------+--------------+
only showing top 20 rows



In [8]:
# Create the products_table DataFrame and drop duplicates. 
products_df = df.select(["product_id", "product_title"]).drop_duplicates()
products_df.show(truncate=False)

+----------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|product_id|product_title                                                                                                                                                                                   |
+----------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|B000JNNWQ2|IRWIN Tools VISE-GRIP Multi Tool Stripper                                                                                                                                                       |
|B000OECQOK|Titan 17005 Stainless Steel Prybar and Scraper Set - 2 Piece                                                                                                        

In [9]:
# Create the review_id_table DataFrame. 
# Convert the 'review_date' column to a date datatype with to_date("review_date", 'yyyy-MM-dd').alias("review_date")
review_id_df = df.select(["review_id","customer_id","product_id","product_parent", to_date("review_date", 'yyyy-MM-dd').alias("review_date")])
review_id_df.show(truncate=False)

+--------------+-----------+----------+--------------+-----------+
|review_id     |customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R2UM5QMHBHC90Q|15785389   |B00H5U9ZD6|115362950     |2015-08-31 |
|RF0D1LEIF6L7  |47910848   |B001TJGCS0|570955425     |2015-08-31 |
|RM6YKIWQVNSY  |36328996   |B000NIK8JW|128843593     |2015-08-31 |
|R1RL3L68ASPS36|51785809   |B008ZYRGUA|407828107     |2015-08-31 |
|R1U4XFBFAG34CY|40757491   |B00K5CA0GC|490746675     |2015-08-31 |
|R3KFIK8P0I91PL|35544833   |B00AIJAA94|148352067     |2015-08-31 |
|RENOAY76PPK1O |16474909   |B00JKEI6K8|331801084     |2015-08-31 |
|RINV884I0NL5V |22601598   |B00AGCHVS6|471514859     |2015-08-31 |
|R5KJH6CXZH2PX |16129808   |B002S0O7U4|162253576     |2015-08-31 |
|RO69JF6QWD0W1 |24382335   |B0084YHXMW|69530650      |2015-08-31 |
|R3L9NQBH3FI55C|49796324   |B00MLSS1SW|916693555     |2015-08-31 |
|R4YH95YPHVU0C |33289687   |B00D4WLS2A|39333316      |2015-08-

In [10]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id","star_rating","helpful_votes","total_votes","vine","verified_purchase"])
vine_df.show(truncate=False)

+--------------+-----------+-------------+-----------+----+-----------------+
|review_id     |star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2UM5QMHBHC90Q|5          |0            |0          |N   |Y                |
|RF0D1LEIF6L7  |4          |0            |0          |N   |Y                |
|RM6YKIWQVNSY  |1          |6            |6          |N   |Y                |
|R1RL3L68ASPS36|4          |0            |0          |N   |Y                |
|R1U4XFBFAG34CY|5          |0            |0          |N   |Y                |
|R3KFIK8P0I91PL|5          |0            |0          |N   |Y                |
|RENOAY76PPK1O |5          |0            |0          |N   |Y                |
|RINV884I0NL5V |1          |0            |0          |N   |Y                |
|R5KJH6CXZH2PX |5          |0            |0          |N   |Y                |
|RO69JF6QWD0W1 |5          |0            |0          |N   |Y    

### Connect to the AWS RDS instance and write each DataFrame to its table. 

In [11]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://database-mod17-challenge.cc57lehqhycf.us-east-2.rds.amazonaws.com:5432/postgres"
config = {"user":"postgres", 
          "password": "DataAnalytics", 
          "driver":"org.postgresql.Driver"}

In [13]:
# Write review_id_df to table in RDS
review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [14]:
# Write products_df to table in RDS
# about 3 min
products_df.write.jdbc(url=jdbc_url, table='products_table', mode=mode, properties=config)

In [15]:
# Write customers_df to table in RDS
# 5 min 14 s
customers_df.write.jdbc(url=jdbc_url, table='customers_table', mode=mode, properties=config)

In [16]:
# Write vine_df to table in RDS
# 11 minutes
vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)